# Notebook to extract LMC Cadence

In [1]:
# Check the version of MAF - the output should be version 2.4.2 or higher.
# 'unknown' means you're running from a git repo version,
# and have made changes to your local code

import lsst.sims.maf
lsst.sims.maf.__version__

'2.13.0.sims-59-g0a27faef'

In [2]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# import our python modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

from lsst.sims.maf.db import ResultsDb

In [4]:
import sys
print(sys.path)

['/Users/dagoret/MacOSX/GitHub/LSST/MyCadence/ExtractLMC', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/obs_lsst/19.0.0-80-g0bd2bb3+1/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/ts_dateloc/master-g60a36c8d46+46/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/ts_observatory_model/master-g3d307a5f45+46/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/sims_almanac/master-g021b69e146+1/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/sims_downtimeModel/master-g55f72efa65+17/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/sims_w_2020_31/stack/miniconda3-py37_4.8.2-1a1d771/DarwinX86/sims_cloudModel/master-ge3724df529+19/python', '/cvmfs/sw.lsst.eu/darwin-x86_64/lsst_sims/s

### Specify my other python path (my installation)

In [5]:
otherpythonpaths=['/Users/dagoret/anaconda3/lib/python37.zip', '/Users/dagoret/anaconda3/lib/python3.7', '/Users/dagoret/anaconda3/lib/python3.7/lib-dynload', '', '/Users/dagoret/.local/lib/python3.7/site-packages', '/Users/dagoret/anaconda3/lib/python3.7/site-packages', '/Users/dagoret/anaconda3/lib/python3.7/site-packages/aeosa', '/Users/dagoret/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/Users/dagoret/.ipython']

In [6]:
for thepath in otherpythonpaths:
    sys.path.append(thepath) # go to parent dir

In [7]:
%load_ext autoreload
%autoreload 2

## astroquery

In [8]:
import astroquery
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle

In [9]:
from astroquery.simbad import Simbad
result_table = Simbad.query_object("lmc")
result_table.pprint(show_unit=True)

MAIN_ID      RA        DEC    ... COO_QUAL COO_WAVELENGTH     COO_BIBCODE    
          "h:m:s"    "d:m:s"  ...                                            
-------- ---------- --------- ... -------- -------------- -------------------
NAME LMC 05 23 34.6 -69 45 22 ...        D              O 2003A&A...412...45P


In [10]:
Angle(result_table["RA"][0]+" hour")

<Angle 5.39294444 hourangle>

In [11]:
Angle(result_table["DEC"][0]+" degrees")

<Angle -69.75611111 deg>

In [12]:
galaxy_coord=SkyCoord(ra=result_table["RA"][0]+"hour",dec=result_table["DEC"][0]+ "degrees")
galaxy_coord

<SkyCoord (ICRS): (ra, dec) in deg
    (80.89416667, -69.75611111)>

In [13]:
galaxy_ra=galaxy_coord.ra.deg
galaxy_dec=galaxy_coord.dec.deg

In [14]:
print(galaxy_ra,galaxy_dec)

80.89416666666666 -69.75611111111111


In [15]:
galaxy_delta_ra=10.75
galaxy_delta_dec=9.17

# Config

In [16]:
filters = ['u', 'g', 'r', 'i', 'z', 'y']
colors = {'u': 'cyan', 'g': 'g', 'r': 'y','i': 'r', 'z': 'm', 'y': 'k'}
Nfilt=len(filters)

In [17]:
nyears=1.0
maxnights=365.25*nyears

## SqL

In [18]:
raCol = 'fieldRA'
decCol = 'fieldDec'

In [19]:
sqlWhere = \
    'fieldRA < %f and ' \
    'fieldRA > %f and ' \
    'fieldDEC < %f and ' \
    'fieldDEC > %f  ' \
     % (galaxy_ra+ galaxy_delta_ra/2., galaxy_ra- galaxy_delta_ra/2.,galaxy_dec+galaxy_delta_dec/2.,galaxy_dec-galaxy_delta_dec/2)

In [20]:
#sqlWhere = \
#    'ra < %f and ' \
#    'ra > %f and ' \
#    'dec < %f and ' \
#    'dec > %f  ' \
#     % (galaxy_ra+ galaxy_delta_ra/2., galaxy_ra- galaxy_delta_ra/2.,galaxy_dec+galaxy_delta_dec/2.,galaxy_dec-galaxy_delta_dec/2)

In [21]:
sqlWhere

'fieldRA < 86.269167 and fieldRA > 75.519167 and fieldDEC < -65.171111 and fieldDEC > -74.341111  '

In [22]:
#sqlWhere = "night < 10"

In [23]:
sqlNo =  " "

## Metric

In [24]:
metric=metrics.PassMetric(cols=['observationId',
                                'night',
                                'observationStartMJD' ,
                                'Field_fieldId',
                                'fiveSigmaDepth',
                                'filter',
                                'airmass',
                                'skyBrightness',
                                'seeingFwhmEff'])

In [40]:
metric=metrics.PassMetric(cols=['observationId','night','observationStartMJD',
                               'fiveSigmaDepth',
                                'filter',
                                'airmass',
                                'skyBrightness','seeingFwhmEff','fieldId','fieldRA','fieldDec'])

# Simu data

In [41]:
opsdb = db.OpsimDatabase('/Users/dagoret/DATA/Cadence/baseline2018a.db')
outDir = 'output_galaxycadence'
resultsDb = db.ResultsDb(outDir=outDir)

# Slicer

In [42]:
slicer_uni=slicers.UniSlicer()
#slicer_point = slicers.UserPointsSlicer(galaxy_ra, galaxy_dec) 
slicer_point = slicers.UserPointsSlicer(galaxy_ra, galaxy_dec, lonCol='fieldRA', latCol='fieldDec', latLonDeg='degrees')
nside = 512
raCol = 'fieldRA'
decCol = 'fieldDec'
slicer_heal = slicers.HealpixSlicer(nside=nside, lonCol=raCol, latCol=decCol)

Healpix slicer using NSIDE=512, approximate resolution 6.870973 arcminutes


# Bundle

- Bundles dict for all filter

In [43]:
bundles = {}
bundles["LMC"] = metricBundles.MetricBundle(metric, slicer_uni, sqlWhere)
print(bundles["LMC"].dbCols)

{'filter', 'seeingFwhmEff', 'observationStartMJD', 'fieldRA', 'night', 'fieldId', 'fiveSigmaDepth', 'airmass', 'fieldDec', 'observationId', 'skyBrightness'}


In [44]:
bgroup = metricBundles.MetricBundleGroup(bundles, opsdb, outDir=outDir,resultsDb=resultsDb)

In [45]:
bgroup.runAll()
bgroup.plotAll(closefigs=False)

Querying database SummaryAllProps with constraint fieldRA < 86.269167 and fieldRA > 75.519167 and fieldDEC < -65.171111 and fieldDEC > -74.341111   for columns ['filter', 'observationStartMJD', 'seeingFwhmEff', 'fieldRA', 'night', 'fieldDec', 'fiveSigmaDepth', 'airmass', 'fieldId', 'observationId', 'skyBrightness']
Found 758 visits
Running:  ['LMC']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Plotting figures with "fieldRA < 86.269167 and fieldRA > 75.519167 and fieldDEC < -65.171111 and fieldDEC > -74.341111  " constraint now.
Plotting complete.


In [46]:
print(bundles["LMC"].dbCols)

{'filter', 'seeingFwhmEff', 'observationStartMJD', 'fieldRA', 'night', 'fieldId', 'fiveSigmaDepth', 'airmass', 'fieldDec', 'observationId', 'skyBrightness'}


In [55]:
print(len(bundles["LMC"].metricValues[0]))

758


In [53]:
bundles["LMC"].metricValues[0]['observationId']

array([   8476,   10032,   10160,   10161,   10165,   10166,   12540,
         12541,   12543,   12552,   12553,   16592,   16593,   17244,
         17245,   17263,   18140,   18141,   18142,   18831,   20378,
         20380,   21908,   21909,   22706,   22708,   25613,   27484,
         27500,   27501,   27502,   27503,   34009,   34011,   34021,
         34022,   34712,   34713,   37554,   37560,   40330,   40331,
         40332,   40333,   40340,   40937,   40948,   40949,   40950,
         40951,   59467,   59468,   59469,   59480,   59481,   60253,
         60254,   60255,   60259,   60260,   75308,   75309,   75310,
         75311,  218076,  230182,  230183,  230184,  230185,  235004,
        235005,  244084,  244086,  244087,  244088,  244089,  244985,
        245737,  245738,  245739,  249904,  249905,  249906,  254167,
        254900,  254901,  254902,  256212,  256213,  256214,  256215,
        256216,  262456,  262457,  262458,  262459,  262460,  263111,
        263126,  263